In [5]:
pip install sklearn.model_selection

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sklearn.model_selection (from versions: none)
ERROR: No matching distribution found for sklearn.model_selection

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
import torch

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www.\S+', '', text)
    # Remove special characters and emojis
    text = re.sub(r'[^a-zA-Z0-9\u4e00-\u9fff\s]', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
# Load the data
data = pd.read_csv('/path/to/your/file.csv')

# Inspect the first few rows
print(data.head())

In [ ]:




# Apply the cleaning function
data['cleaned_text'] = data['text'].astype(str).apply(clean_text)

In [ ]:


label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['party_support'])  # 'party_support' should be the column with labels

In [ ]:


# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

# Tokenize the text
data['tokens'] = data['cleaned_text'].apply(lambda x: tokenizer.encode_plus(
    x,
    add_special_tokens=True,
    max_length=128,  # adjust based on sentence length distribution
    padding='max_length',
    truncation=True,
    return_tensors='pt'  # PyTorch format
))

In [ ]:


# Convert token IDs and attention masks to tensors
input_ids = torch.cat([item['input_ids'] for item in data['tokens']], dim=0)
attention_masks = torch.cat([item['attention_mask'] for item in data['tokens']], dim=0)
labels = torch.tensor(data['label'].values)

In [ ]:

train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, test_size=0.1)
train_masks, val_masks = train_test_split(attention_masks, test_size=0.1)

In [ ]:

batch_size = 32

# Create DataLoader for training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create DataLoader for validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)